In [2]:
from policyengine_us import Microsimulation
from policyengine_us.data import PUF_2022

sim = Microsimulation(dataset=PUF_2022)

In [4]:
sim.calculate("income_tax", 2024).sum() / 1e9

2343.3519753471714